<center><img src='https://drive.google.com/uc?id=1_utx_ZGclmCwNttSe40kYA6VHzNocdET' height="60"></center>

AI TECH - Akademia Innowacyjnych Zastosowań Technologii Cyfrowych. Program Operacyjny Polska Cyfrowa na lata 2014-2020
<hr>

<center><img src='https://drive.google.com/uc?id=1BXZ0u3562N_MqCLcekI-Ens77Kk4LpPm'></center>

<center>
Projekt współfinansowany ze środków Unii Europejskiej w ramach Europejskiego Funduszu Rozwoju Regionalnego
Program Operacyjny Polska Cyfrowa na lata 2014-2020,
Oś Priorytetowa nr 3 "Cyfrowe kompetencje społeczeństwa" Działanie  nr 3.2 "Innowacyjne rozwiązania na rzecz aktywizacji cyfrowej"
Tytuł projektu:  „Akademia Innowacyjnych Zastosowań Technologii Cyfrowych (AI Tech)”
    </center>

Code based on https://github.com/pytorch/examples/blob/master/mnist/main.py

In this exercise we are using high level abstractions from torch.nn like nn.Linear.
Note: during the next lab session we will go one level deeper and implement more things
with bare hands.

Tasks:

    1. Read the code.

    2. Check that the given implementation reaches 95% test accuracy for architecture input-128-128-10 after few epochs.

    3. Add the option to use SGD with momentum instead of ADAM.

    4. Experiment with different learning rates, plot the learning curves for different
    learning rates for both ADAM and SGD with momentum.

    5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
    Note that this requires creating a list of layers as an atribute of the Net class,
    and one can't use a standard python list containing nn.Modules (why?).
    Check torch.nn.ModuleList.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self, hidden_layers_sizes=[128, 128]):
        super(Net, self).__init__()
        # After flattening an image of size 28x28 we have 784 inputs
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(784, hidden_layers_sizes[0]))
        self.layers.extend([nn.Linear(hidden_layers_sizes[i], hidden_layers_sizes[i+1]) for i in range(len(hidden_layers_sizes)-1)])
        self.layers.append(nn.Linear(hidden_layers_sizes[-1], 10))
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        for f in self.layers:
            x = f(x)
            x = F.relu(x)
        output = F.log_softmax(x, dim=1)
        return output


def train(model, device, train_loader, optimizer, epoch, log_interval):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return loss.item()


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [3]:
batch_size = 256
test_batch_size = 1000
epochs = 5
lr = 1e-2
seed = 1
log_interval = 10
use_cuda = torch.cuda.is_available()

In [4]:
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

In [5]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [6]:
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, log_interval)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304566
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.457628
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.240940
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.176437
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.132400
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.205209
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.088946
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.987405
Train Epoch: 1 [20480/60000 (34%)]	Loss: 1.144178
Train Epoch: 1 [23040/60000 (38%)]	Loss: 1.145256
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.989151
Train Epoch: 1 [28160/60000 (47%)]	Loss: 1.107859
Train Epoch: 1 [30720/60000 (51%)]	Loss: 1.026872
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.947113
Train Epoch: 1 [35840/60000 (60%)]	Loss: 1.110440
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.075348
Train Epoch: 1 [40960/60000 (68%)]	Loss: 1.099263
Train Epoch: 1 [43520/60000 (72%)]	Loss: 1.139194
Train Epoch: 1 [46080/60000 (77%)]	Loss: 1.036064
Train Epoch: 1 [48640/60000 (81%)]	Loss: 1.020343
Train Epoc

In [7]:
 # 3. Add the option to use SGD with momentum instead of ADAM.
momentum = 0.9
model_with_momentum = Net().to(device)
optimizer = optim.SGD(model_with_momentum.parameters(), lr=lr, momentum=momentum)

for epoch in range(1, epochs + 1):
    train(model_with_momentum, device, train_loader, optimizer, epoch, log_interval)
    test(model_with_momentum, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.299955
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.275397
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.202121
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.015061
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.650691
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.469198
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.060607
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.789240
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.834984
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.700384
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.617244
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.650315
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.688859
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.624487
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.605043
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.589915
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.564833
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.508754
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.593077
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.469831
Train Epoc

In [8]:
### Your code goes here ###
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=DHUSDP5H5QRF6ZJGXYAS
%env CLEARML_API_SECRET_KEY=w7yowOv0p7WHKK4tU3AlEYNfG1gNU80jx2XwS0z3w9XaSaMnPR
###########################

from clearml import Task
task = Task.init(project_name="michal_mierzejewski_lab4", task_name="pytorch")

env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=DHUSDP5H5QRF6ZJGXYAS
env: CLEARML_API_SECRET_KEY=w7yowOv0p7WHKK4tU3AlEYNfG1gNU80jx2XwS0z3w9XaSaMnPR
ClearML Task: created new task id=2b638735f84145d09710dd43d19e7488
2023-11-16 13:02:29,715 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/3d1e60ba8ae5463ead62d15e76117409/experiments/2b638735f84145d09710dd43d19e7488/output/log


In [ ]:
# 4. Experiment with different learning rates, plot the learning curves for different
# learning rates for both ADAM and SGD with momentum.
import matplotlib.pyplot as plt
grid = {'lr': [1e-4]}#, 1e-3, 1e-2, 1e-1, 1]}
for lr in grid['lr']:
    losses = []
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(1, epochs + 1):
        losses.append(train(model, device, train_loader, optimizer, epoch, log_interval))
        test(model, device, test_loader)
    print(losses)
    plt.plot(losses, label=f'lr={lr}')
    plt.show()
    
        
for lr in grid['lr']:
    losses = []
    model_with_momentum = Net().to(device)
    optimizer = optim.SGD(model_with_momentum.parameters(), lr=lr, momentum=momentum)
    for epoch in range(1, epochs + 1):
        losses.append(train(model_with_momentum, device, train_loader, optimizer, epoch, log_interval))
        test(model_with_momentum, device, test_loader)
    print(losses)
    plt.plot(losses, label=f'lr={lr}')
    plt.show()


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310177
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.261712
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.179881
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.113709
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.922168


In [ ]:
"""5. Parameterize the constructor by a list of sizes of hidden layers of the MLP.
Note that this requires creating a list of layers as an atribute of the Net class,
and one can't use a standard python list containing nn.Modules (why?).
Check torch.nn.ModuleList."""
